<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2022notebooks/2022_0625DETR_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DETR を用いた領域切り出し

ここでは，バウンディングボックス，画像から注目する領域の四角形を切り出しと，パノプティック切り出しを行う。

[DETR](https://arxiv.org/pdf/2005.12872) とは，トランスフォーマーを用いた符号化器-復号化器 (encoder-decoder) モデルである。


<center>
<img src="https://komazawa-deep-learning.github.io/2022assets/2020Carion_DETR_fig2ja.svg" width="88%"><br/>
</center>


## 準備作業

In [ ]:
%config InlineBackend.figure_format = 'retina'
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

from termcolor import colored
import platform
HOSTNAME = platform.node().split('.')[0]

import os
HOME = os.environ['HOME']

try:
    import ipynbname
except ImportError:
    !pip install ipynbname > /dev/null
import ipynbname
FILEPATH = str(ipynbname.path()).replace(HOME+'/','')

import pwd
USER=pwd.getpwuid(os.geteuid())[0]

from datetime import date
TODAY=date.today()

import torch
TORCH_VERSION = torch.__version__

color = 'green'
print('日付:',colored(f'{TODAY}', color=color, attrs=['bold']))
print('HOSTNAME:',colored(f'{HOSTNAME}', color=color, attrs=['bold']))
print('ユーザ名:',colored(f'{USER}', color=color, attrs=['bold']))
print('HOME:',colored(f'{HOME}', color=color,attrs=['bold']))
print('ファイル名:',colored(f'{FILEPATH}', color=color, attrs=['bold']))
print('torch.__version__:',colored(f'{TORCH_VERSION}', color=color, attrs=['bold']))

In [ ]:
from PIL import Image
import requests
import io
import math
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:    
    !pip install japanize_matplotlib
import japanize_matplotlib

import torch
from torch import nn
#from torchvision.models import resnet50
import torchvision.transforms as T
import numpy
torch.set_grad_enabled(False);

パノプティック 切り出しのための API をインストール

In [ ]:
try:
    import panopticapi
except ImportError:
    !pip install git+https://github.com/cocodataset/panopticapi.git --upgrade

import panopticapi
from panopticapi.utils import id2rgb, rgb2id

In [ ]:
# MS COCO のクラス
CLASSES = [
    'N/A', '人', '自転車', '車', 'バイク', '飛行機', 'バス',
    '電車', 'トラック', 'ボート', '信号機', '消火栓', 'N/A',
    '停止サイン', '駐車メータ', 'ベンチ', '鳥', 'ネコ', 'イヌ', '馬',
    '羊', '牛', '象', '熊', 'シマウマ', 'キリン', 'N/A', 'バックパック',
    '傘', 'N/A', 'N/A', 'ハンドバッグ', 'ネクタイ', 'スーツケース', 'フリスビー', 'スキー',
    'スノーボード', 'sports ball', '凧', '野球のバット', '野球のグラブ',
    'スケートボード', 'サーフボード', 'テニスラケット', 'ボトル', 'N/A', 'ワイングラス',
    'カップ', 'フォーク', 'ナイフ', 'スプーン', 'ボウル', 'バナナ', 'りんご', 'サンドウィッチ',
    'オレンジ', 'ブロッコリ', 'ニンジン', 'ホットドッグ', 'ピザ', 'ドーナッツ', 'ケーキ',
    'イス', 'ソファ', '鉢植え', 'ベッド', 'N/A', 'ダイニングテーブル', 'N/A',
    'N/A', 'トイレ', 'N/A', 'テレビ', 'ラップトップ', 'マウス', 'リモコン', 'キーボード',
    '携帯電話', '電子レンジ', 'コンロ', 'トースター', '洗面台', '冷蔵庫', 'N/A',
    '本', '時計', '花瓶', 'ハサミ', 'テディベア', 'ドライヤー',
    '歯ブラシ'
]


# 視覚化のための色定義
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# 平均と標準偏差を用いて入力画像を正規化
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# バウンディングボックスの前処理。
# バウンディングボックスの中心座標と幅から，左，上，右，下座標を計算する
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b


def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()
    
# Detectron2 と クラスの定義が異なるため，その差異を吸収するため
coco2d2 = {}
count = 0
for i, c in enumerate(CLASSES):
    if c != "N/A":
        coco2d2[i] = count
    count+=1

# 唯一の前処理 平均を引いて標準偏差で割る標準化
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## モデルハブから訓練済の結合係数を取得


In [ ]:
detr_resnet = torch.hub.load(repo_or_dir='facebookresearch/detr', 
                             model='detr_resnet50', 
                             pretrained=True)

detr_resnet_panoptic, postprocessor0 = torch.hub.load(
    repo_or_dir='facebookresearch/detr', 
    model='detr_resnet50_panoptic', 
    pretrained=True,
    return_postprocessor=True,
    num_classes=250)


COCO 検証用データセットから画像を取得

In [ ]:
# 直下行の 画像 URL を書き換えて実習すること，場合によっては URL に特殊文字が含まれている場合がある。
# そのときには URL 全体を 引用記号で囲んで指定する。例えば 'https://hogehoge.jpg' のようにする。
!wget http://images.cocodataset.org/val2017/000000039769.jpg -O sample.jpg
im = Image.open('sample.jpg')
IPython.display.Image('sample.jpg')

In [ ]:
# 旧コード，変更すると動作しなかったので，上のセルに書き換えた
# if not os.path.exists('000000039769.jpg'):
#     !wget http://images.cocodataset.org/val2017/000000039769.jpg
# im = Image.open('000000039769.jpg')
# IPython.display.Image('000000039769.jpg')

* 前処理を，各データに施し，予測をフィルタリング
* クラスの信頼度が しきい値 (0.9) よりも高い物体のみを保存する
 (非オブジェクトの予測は除外)。
* より多くの予測を得たい場合は，この閾値を下げる


In [ ]:
# 平均を引いて，標準偏差で割る正規化をバッチサイズ 1 の入力画像に対して行う
img = transform(im).unsqueeze(0)
outputs = detr_resnet(img) # model に通して出力を得る

# 確信度 0.9 以上の予測値だけを考慮する。
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
keep = probas.max(-1).values > 0.9

# バウンディングボックス(注目する画像の矩形領域) を縮尺に合わせて拡大
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

In [ ]:
plot_results(im, probas[keep], bboxes_scaled) # 結果の描画

## パノプティック切り出し

In [ ]:
# 直下行の 画像 URL を書き換えて実習すること，場合によっては URL に特殊文字が含まれている場合がある。
# そのときには URL 全体を 引用記号で囲んで指定する。例えば 'https://hogehoge.jpg' のようにする。
!wget http://images.cocodataset.org/val2017/000000281759.jpg -O sample2.jpg
im = Image.open('sample2.jpg')
IPython.display.Image('sample2.jpg')

In [ ]:
# import os
# img_fname = '000000281759.jpg'
# if not os.path.exists(img_fname):
#     !wget http://images.cocodataset.org/val2017/000000281759.jpg -O 000000281759.jpg
# im = Image.open(img_fname)
# IPython.display.Image(img_fname)

検出の実施

In [ ]:
img = transform(im).unsqueeze(0)
out = detr_resnet_panoptic(img)

/root/.cache/torch/hub/facebookresearch_detr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


これは各クエリのマスクを返すので、信頼度の高いものを可視化してみましょう。
<!-- This returns a mask for each query, let us visualize the high confidence ones -->

In [ ]:
#  "no-object" クラス（最後の 1 つ）を除いたスコアを計算する
scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]

# 信頼性のしきい値を設定
keep = scores > 0.85

# しきい値以上のマスクを表示する
ncols = 5
fig, axs = plt.subplots(ncols=ncols, nrows=math.ceil(keep.sum().item() / ncols), figsize=(18, 10))
for line in axs:
    for a in line:
        a.axis('off')
for i, mask in enumerate(out["pred_masks"][keep]):
    ax = axs[i // ncols, i % ncols]
    ax.imshow(mask, cmap="cividis")
    ax.axis('off')
fig.tight_layout()

個々のマスクが揃ったので、予測を統合して統一された パノラマ的 (パノプティックな) 切り出しを行えます。
そのために DETR の後処理ルーチンを使用します。

In [ ]:
# 後処理ルーチンは予測値のターゲットサイズと同じサイズ (ここでは画像サイズに設定) の入力画像である必要があります。
result = postprocessor0(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]

簡単な結果の表示

In [ ]:
import itertools
import seaborn as sns
palette = itertools.cycle(sns.color_palette())

# 切り出し結果は png 画像フォーマットとして保存される
panoptic_seg = Image.open(io.BytesIO(result['png_string']))
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8).copy()

# 各マスクに対応する id を取得
panoptic_seg_id = rgb2id(panoptic_seg)

# 最後に各マスクを個別に着色する
panoptic_seg[:, :, :] = 0
for id in range(panoptic_seg_id.max() + 1):
    panoptic_seg[panoptic_seg_id == id] = numpy.asarray(next(palette)) * 255
plt.figure(figsize=(15,15))
plt.imshow(panoptic_seg)
plt.axis('off')
plt.show()

## Detectron2 を用いた視覚化

In [ ]:
try:
    import detectron2
except ImportError:
    !pip install 'git+https://github.com/facebookresearch/detectron2.git'

import detectron2
detectron2.__version__

In [ ]:
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
if isColab:
    from google.colab.patches import cv2_imshow

結果の表示

In [ ]:
from copy import deepcopy

# We extract the segments info and the panoptic result from DETR's prediction
segments_info = deepcopy(result["segments_info"])

# Panoptic predictions are stored in a special format png
panoptic_seg = Image.open(io.BytesIO(result['png_string']))
final_w, final_h = panoptic_seg.size

# We convert the png into an segment id map
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8)
panoptic_seg = torch.from_numpy(rgb2id(panoptic_seg))

    
# Detectron2 uses a different numbering of coco classes, here we convert the class ids accordingly
meta = MetadataCatalog.get("coco_2017_val_panoptic_separated")
for i in range(len(segments_info)):
    c = segments_info[i]["category_id"]
    segments_info[i]["category_id"] = meta.thing_dataset_id_to_contiguous_id[c] if segments_info[i]["isthing"] else meta.stuff_dataset_id_to_contiguous_id[c]

    
# Finally we visualize the prediction
v = Visualizer(numpy.array(im.copy().resize((final_w, final_h)))[:, :, ::-1], meta, scale=1.0)
v._default_font_size = 20
v = v.draw_panoptic_seg_predictions(panoptic_seg, segments_info, area_threshold=0)
if isColab:
    cv2_imshow(v.get_image())
else:
    plt.figure(figsize=(20,20))
    plt.imshow(v.get_image()[:,:,::-1])
    plt.show()